In [32]:
import json
import os
import subprocess
from openai import OpenAI

In [33]:
# Get dataset directory
directory = os.getcwd()
dataset_directory = os.path.join(directory, 'problems_dataset')

In [34]:
#Function to get all the filenames of dataset problems
def get_problem_filenames():
    filenames = []
    for filename in os.listdir(dataset_directory):
        if filename.endswith('.json'):
            filenames.append(filename)
    return filenames

In [35]:
# Function to load a problem file from dataset directory
def load_problem(problem_filename):
    path = os.path.join(dataset_directory, problem_filename)
    with open(path, 'r') as json_file:
        problem = json.load(json_file)
    return problem

In [36]:
# Custom exception classes with error messages
class CompilationError(Exception):
    def __init__(self, message):
        self.message = message
        super().__init__(self.message)

class RuntimeError(Exception):
    def __init__(self, message):
        self.message = message
        super().__init__(self.message)

class TestFailed(Exception):
    def __init__(self, message, input, expected_output, output):
        self.message = message
        super().__init__(self.message)

class TestSuccess(Exception):
    pass

In [37]:
# Function to compile the C++ program
def compile_cpp(source_file, exec_file):
    compile_command = ["g++", source_file, "-o", exec_file]
    result = subprocess.run(compile_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if result.returncode != 0:
        raise CompilationError(f"Compilation failed: {result.stderr.decode('utf-8')}")
    return True

In [38]:
# Function to run the compiled program with the provided input and capture the output
def run_program(executable, input_data):
    process = subprocess.Popen([f"./{executable}"], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output, error = process.communicate(input=input_data.encode())
    if process.returncode != 0:
        raise RuntimeError(f"Runtime error: {error.decode('utf-8')}")
    return output.decode('utf-8')

In [40]:
# Function to run tests and compare output
def run_tests(test_cases, executable):
    f = open('tmp_output.txt', 'w')
    for i, test in enumerate(test_cases):
        input_data = test["input"]
        expected_output = test["output"]
        try:
            actual_output = run_program(executable, input_data)
        except RuntimeError as e:
            print(f"Test case {i+1}: RUNTIME ERROR")
            print(e.message)
            continue

        # Compare outputs
        if actual_output.strip() == expected_output.strip():
            f.write(f"Test case {i+1}: PASSED\n")
        else:
            f.write(f"Test case {i+1}: FAILED\n")
            f.write(f"Expected output:\n{expected_output}\n")
            f.write(f"Actual output:\n{actual_output}\n")
            raise TestFailed(f"Test case {i+1} failed", input_data, expected_output, actual_output)
    f.close()
    raise TestSuccess("All test cases passed") 

In [41]:
# Function to parse test cases from the problem JSON
def parse_test_cases(problem):
    test_cases = []
    for test_case in problem["public_tests"]:
        test_cases.append({
            "input": test_case["input"],
            "output": test_case["output"]
        })
    for test_case in problem["generated_tests"]:
        test_cases.append({
            "input": test_case["input"],
            "output": test_case["output"]
        })
    return test_cases

In [42]:
# Function to test solution
def test_solution(problem, solution_code):
    # Write the solution to a file
    source_file = "solution.cpp"
    with open(source_file, "w") as file:
        file.write(solution_code)
    
    # Compile the solution
    executable = "solution"
    compile_cpp(source_file, executable)
    
    # Parse test cases
    test_cases = parse_test_cases(problem)
    
    # Run the tests
    try:
        run_tests(test_cases, executable)
    except TestFailed as e:
        raise TestFailed(e.message, e.input, e.expected_output, e.output)
    except TestSuccess:
        raise TestSuccess("All test cases passed")
    except Exception as e:
        exit(e.message)
    finally:
        # Delete the source file and executable
        os.remove(source_file)
        os.remove(executable)

In [43]:
# Mock solution
solution_code = "#include<iostream> \nusing namespace std; \nint main() { \nint a, b; \ncin >> a >> b; \ncout << a + b << endl; \nreturn 0; \n} \n"
# Load addition.json problem
with open('addition.json', 'r') as json_file:
    problem = json.load(json_file)

try:
    test_solution(problem, solution_code)
except TestFailed as e:
    print(e.message)
    print("Input:\n", e.input)
    print("Expected output:\n", e.expected_output)
    print("Actual output:\n", e.output)
except TestSuccess:
    print("All test cases passed")


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [8]:
client = OpenAI()

# Function to generate a solution using OpenAI's GPT-3.5
def generate_solution(problem):
    title = problem['title']
    description = problem['description']
    input_desc = problem['input']
    output_desc = problem['output']

    # Create a prompt based on the problem description
    prompt = f"Problem: {title}\nDescription: {description}\nInput: {input_desc}\nOutput: {output_desc}\n\nWrite a Python function to solve the problem."

    # Call OpenAI API to generate the solution
    response = client.chat.completions.create(model="gpt-3.5-turbo",
    messages=[
           {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
    ])

    # Extract the solution from the API response
    solution_code = response.choices[0].message.content.strip()
    return solution_code

In [9]:
# Function to test the generated solution
def test_solution(solution_code, test_cases):
    # Define a local environment to execute the solution code
    local_env = {}
    exec(solution_code, local_env)

    # Extract the function name (assuming it's the first function defined in the code)
    function_name = list(local_env.keys())[0]

    # Test each case
    for case in test_cases:
        input_values = list(map(int, case['input'].split()))
        expected_output = int(case['output'])

        # Call the generated function with test case inputs
        result = local_env[function_name](*input_values)

        print(f"Input: {case['input']}, Expected Output: {expected_output}, Obtained Output: {result}")
        assert result == expected_output, "Test case failed!"

    print("All test cases passed!")

In [11]:
# Solve each problem in the dataset
for problem in dataset['problems']:
    solution_code = generate_solution(problem)
    print(f"Generated Solution:\n{solution_code}\n")
    # test_solution(solution_code, problem['test_cases'])

Generated Solution:
Here's a Python function that solves the problem:

```python
def sum_of_two_numbers(a, b):
    return a + b

# Input
a = int(input("Enter the first integer (a): "))
b = int(input("Enter the second integer (b): "))

# Output
result = sum_of_two_numbers(a, b)
print("Sum of", a, "and", b, "is:", result)
```

You can call this function with two integers as input, and it will output their sum. Let me know if you need any further assistance.

